In [1]:
import sys
import os
__file__ = %pwd
sys.path.append(os.path.dirname(__file__))
from Package import *
logger = logging.getLogger()
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.WARNING)
print(__file__)

/Project/Local_Project/InceptionTimeV2.8/CODE/demo


In [2]:
sub_datasets = datasets[:10]
train_epoch = 1000
attack_epoch1 = 1000
attack_epoch2 = 100
override = False

In [3]:
method_path = ''
for i in sub_datasets:
    trainer = Trainer(
        i, 
        batch_size=256, 
        epoch = train_epoch, 
        override=override,
        continue_train = False,
        )
    method_path = trainer.method_path
    # trainer.train_and_evaluate()

In [5]:
concat_metrics_train(mode="train", method=method_path, datasets=sub_datasets)
concat_metrics_train(mode="test", method=method_path, datasets=sub_datasets)

In [4]:
keys=[]
for k in sys.modules.keys():
    if "CODE.attack.mix" in k:
        print(k)
        keys.append(k)
for k in keys:
    del sys.modules[k]

from CODE.attack.mix import Mix
model_class = Mix

In [5]:
attack_series = [
    {
        'epoch': attack_epoch1,
        'swap': True,
        'kl_loss': True,
        'CW': True,
        'sign_only': False,
        'alpha': 0.02
    },
    {
        'epoch': attack_epoch1,
        'swap': False,
        'kl_loss': False,
        'CW': False,
        'sign_only': False,
        'alpha': 0.02
    },
    {
        'epoch': 1,
        'swap': False,
        'kl_loss': False,
        'CW': False,
        'sign_only': True,
        'alpha': 0.1
    },
]

In [6]:
path_parameter = [
    "swap",
    "kl_loss",
    "CW",
    "sign_only"
]

In [7]:
attack_method_path_series = []

for i in sub_datasets:
    for j in attack_series:
        # pprint(j)
        attacker = model_class(
            dataset=i,
            batch_size=128, 
            epoch=j['epoch'], 
            swap=j['swap'],
            kl_loss=j['kl_loss'],
            CW=j['CW'],
            train_method_path = method_path,
            adeversarial_training = True,
            sign_only=j['sign_only'],
            alpha=j['alpha'],
            path_parameter=path_parameter
            )
        attack_method_path_series.append(attacker.attack_method_path) if not attacker.attack_method_path in attack_method_path_series else None
        # attacker.perturb_all(
        #     to_device=False,
        #     override=override,
        # )
        # attacker.build_adeversarial_training_data()
        # pprint(attacker.path_parameter)


In [8]:
attack_method_path_series

['swap=True_kl_loss=True_CW=True_sign_only=False',
 'swap=False_kl_loss=False_CW=False_sign_only=False',
 'swap=False_kl_loss=False_CW=False_sign_only=True']

In [9]:
train_method_path_series = []
method_path = ''
for j in attack_method_path_series:
    for i in sub_datasets:
        trainer = Trainer(
            i, 
            batch_size=256,
            epoch = train_epoch,
            override=override,
            continue_train = False,
            adeversarial_training = True,
            adeversarial_path=j,
            adeversarial_resume=False
            )
        method_path = trainer.method_path
        train_method_path_series.append(method_path) if not method_path in train_method_path_series else None
        trainer.train_and_evaluate()
    concat_metrics_train(mode="train", method=method_path, datasets=sub_datasets)
    concat_metrics_train(mode="test", method=method_path, datasets=sub_datasets)

Epoch 00301: reducing learning rate of group 0 to 5.0000e-04.
Epoch 00352: reducing learning rate of group 0 to 2.5000e-04.
Epoch 00403: reducing learning rate of group 0 to 1.2500e-04.
Epoch 00454: reducing learning rate of group 0 to 1.0000e-04.
Epoch 00341: reducing learning rate of group 0 to 5.0000e-04.
Epoch 00629: reducing learning rate of group 0 to 2.5000e-04.
Epoch 00933: reducing learning rate of group 0 to 1.2500e-04.
Epoch 00206: reducing learning rate of group 0 to 5.0000e-04.
Epoch 00257: reducing learning rate of group 0 to 2.5000e-04.
Epoch 00308: reducing learning rate of group 0 to 1.2500e-04.
Epoch 00359: reducing learning rate of group 0 to 1.0000e-04.
Epoch 00162: reducing learning rate of group 0 to 5.0000e-04.
Epoch 00213: reducing learning rate of group 0 to 2.5000e-04.
Epoch 00264: reducing learning rate of group 0 to 1.2500e-04.
Epoch 00479: reducing learning rate of group 0 to 1.0000e-04.
Epoch 00180: reducing learning rate of group 0 to 5.0000e-04.
Epoch 00

In [13]:
attack_series = [
    {
        'epoch': attack_epoch2,
        'swap': True,
        'kl_loss': True,
        'CW': True,
        'sign_only': False,
        'alpha': 0.02
    },
    {
        'epoch': attack_epoch2,
        'swap': False,
        'kl_loss': False,
        'CW': False,
        'sign_only': False,
        'alpha': 0.02
    },
    {
        'epoch': 1,
        'swap': False,
        'kl_loss': False,
        'CW': False,
        'sign_only': True,
        'alpha': 0.1
    },
]

In [14]:
for j in attack_series:
    for k in train_method_path_series:
        for i in sub_datasets:
            attacker = model_class(
                dataset=i,
                batch_size=128, 
                epoch=j['epoch'], 
                swap=j['swap'],
                kl_loss=j['kl_loss'],
                CW=j['CW'],
                train_method_path = k,
                adeversarial_training = False,
                sign_only=j['sign_only'],
                alpha=j['alpha'],
                path_parameter=path_parameter
                )
            attacker.perturb_all(
                to_device=True,
                override=override,
            )
        concat_metrics_attack(
            method = os.path.join(k, attacker.attack_method_path),datasets=sub_datasets)

/opt/anaconda/anaconda3/envs/tf-gpu/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda/anaconda3/envs/tf-gpu/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda/anaconda3/envs/tf-gpu/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda/anaconda3/envs/tf-gpu/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda/anaconda3/envs/tf-gpu/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda/anaconda3/envs/tf-gpu/lib/python3.10/site-packages/numpy/core/_meth